# Item 46. 함수 타입 파라미터를 갖는 함수에 inline 한정자를 붙여라

## inline 키워드
- inline 키워드는 코드를 호출하는 부분에 코드를 복사해 넣어준다.
- 톱레벨 템플릿 콜백 패턴을 사용할때 인라인 키워드로 정의하면 성능을 향상시킬 수 있다.


In [2]:
fun nonInline(block: () -> Unit) {
    println("1")
    block()
    println("2")
}

inline fun inline(block: () -> Unit) {
    println("1")
    block()
    println("2")
}

nonInline { println("Hello") }
inline { println("Hello") }

true
false


```java
  public static final void main() {
        // 일반 템플릿 콜백은 함수 호출을 한다
      nonInline((Function0)null.INSTANCE);
        // 인라인 템플릿 콜백은 코드를 복사해 넣어준다
      int $i$f$inline = false;
      String var1 = "1";
      System.out.println(var1);
      int var2 = false;
      String var3 = "Hello";
      System.out.println(var3);
      var1 = "2";
      System.out.println(var1);
   }
```

- inline 함수는 타입 아규먼트에 reified 한정자를 붙일 수 있다.
- 람다 실행시 더 빠르게 수행 가능
- 비지역 리턴을 사용할 수 있다.

## reified
- 제네릭은 기본적으로 컴파일시점에 타입소거가되어 자바바이트코드상에 제네릭정보는 남지않는다.
- 따라서 시그니처에 선언된 제네릭은 코드상에서 사용될 수 없다.
- 인라인함수의 경우 컴파일시점에 흑마법을 부리면서 코드를 삽입하는데 이때 reified를 사용하면 제네릭 정보까지 같이 자바바이트코드로 삽입한다.

In [ ]:
inline fun <reified T> printType() {
    val type = T::class.simpleName // reified 를 썼기때문에 T에서 클래스 정보를 가져올 수 있다.
    println(type)
}

printType<Int>()


```java
   int $i$f$printType = false;
      String type$iv = Reflection.getOrCreateKotlinClass(Integer.class).getSimpleName(); // 타입 T가 Integer로 치환되어서 삽입된다.
      System.out.println(type$iv);
```

## 람다가 더 빨리 수행
- 스택을 한단계 덜 쌓고 추적도 덜해야하기 때문에 람다를 사용하는 코드가 더 빠르다.
- 하지만 람다가 아닌경우도 마찬가지
- 그럼에도 람다사용시만 인라인을 사용을 권장하는 이유는? (인텔리제이에서 일반함수 인라인시엔 굳이 쓰지말라한다)  
- 람다가 실제로 동작하는 방식은 람다를 가진 함수형 인터페이스를 직접 구현한 익명 객체를 만들고 그 객체를 호출하는 것이다.
- 즉 inline은 익명 객체 생성 자체를 없애버리므로 상당히 큰 차이를 낳는셈
- 또한 람다는 클로저를 만들면서 캡쳐값도 저장해야하는데 이또한 성능에 영향을 미친다.

## 비지역리턴(non-local return)
- 일반적 템플릿 콜백패턴에서는 일반 return 을 사용할 수 없음 오직 return@label 만 가능
- inline 함수에서는 어차피 호출메서드에 코드로 들어가기때문에 호출메서드의 return 도 사용가능해진다(비지역리턴 가능)

In [ ]:
nonInline {
    println("test")
    return@nonInline
}

inline {
    println("test")
    return@inline
    return // 가능
}

## inline 한정자의 한계
- inline 함수는 코드를 복사해 넣기때문에 코드가 많아지면 컴파일된 바이트코드도 커진다.따라서 inline 함수는 코드가 작을때만 사용하는것이 좋다.
- 같은 논리로 재귀에 사용할 수 없다

## crossinline과 noninline
- 좀더 제약있는 인라인을 만들고싶을때 사용하는 키워드
- crossinline : 아규먼트로 인라인 함수를 받지만, 비지역적 리턴을 하는 함수는 받을수 없게 만든다. 인라인으로 만들지 않은 다른 람다 표현식과 조합해서 사용할때 문제가 있으면 사용
- noinline: 인라인이 되지 않는다


In [ ]:
inline fun inline3(
    crossinline block: () -> Unit,
    noinline block2: () -> Unit,
    block3: () -> Unit
) {
    block()
    block2()
    block3()
}

inline fun <reified T> printType() {
    val type = T::class.simpleName
    println(type)
}

fun main() {
    inline3(
        { printType<Int>() }, // crossinline 이므로 비지역적 리턴을 할 수 없다.
        { printType<Int>() },  // noinline 이므로 인라인이 되지 않는다.
        { return printType<Int>() } // 인라인되고 비지역적 리턴이 가능하다.
    )
}